In [1]:
from nilearn.input_data import NiftiMasker, MultiNiftiMasker
from nilearn.datasets import fetch_atlas_basc_multiscale_2015
from fastsrm.fastsrm import FastSRM
from fastsrm import fastsrm
import matplotlib.pyplot as plt
from nilearn import image
from nilearn import plotting
import os
import glob
import numpy as np
from nilearn.image import new_img_like
import ibc_public
import nibabel as nib

%matplotlib inline

/home/sshankar/env/fastsrm/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
# Specify the mask image
_package_directory = os.path.dirname(os.path.abspath(ibc_public.__file__))
mask_gm = nib.load(os.path.join(_package_directory, '../ibc_data', 'gm_mask_3mm.nii.gz'))

In [169]:
# Task of interest
task = 'raiders'

# Any specific files that should be used for FastSRM
if task == 'clips':
    filepattern = '*Trn*.nii.gz'
else:
    filepattern = '*.nii.gz'

In [170]:
# Do this for a previously unused atlas. 
# Else, you should have a .npy file saved from before, and you can just load it.
# The transform() funtion takes a few minutes to run so don't run it 
# unless you absolutely need to.

# Now, a bit of shape shifting to make the atlas compatible with
# what fastsrm.reduce_data() requires.
# 1. Add a 4th dimension to the 3D atlas. The 4th dimension will have as many
#   elements as atlas parcesl (444, in this case)
# 2. The 3D "volume" pertaining to each 4th dimension will contain 1 in the
#   "voxel" for that parcel and 0 otherwise
# 3. Apply the atlas masker set up previously to transform the new 4D atlas
#   into 2D, with n_voxel rows and n_parcel columns,
#   where n_voxel is the number of voxels in the transformed image matrix
# 4. Reduce the 2D atlas matrix to 1D by using the argmax function along the
#   column dimension. Now, the transformed atlas has n_voxel elements.

atlas_loc = os.path.join('..', task, '3mm')
if os.path.exists(os.path.join(atlas_loc, 'atlas_masked.npy')):
    atlas = np.load(os.path.join(atlas_loc, 'atlas_masked.npy'), allow_pickle=True)
else:
    # Specify the atlas
    basc444 = fetch_atlas_basc_multiscale_2015()['scale444']
    basc_im = image.load_img(basc444).get_data()

    atlas_masker = NiftiMasker(mask_img=mask_gm).fit()

    if len(basc_im.shape) == 3:
        n_components = len(np.unique(basc_im)) - 1
        xa, ya, za = basc_im.shape
        A = np.zeros((xa, ya, za, n_components + 1))
        atlas = np.zeros((xa, ya, za, n_components + 1))
        for c in np.unique(basc_im)[1:].astype(int):
            X_ = np.copy(basc_im)
            X_[X_ != c] = 0.
            X_[X_ == c] = 1.
            A[:, :, :, c] = X_
        atlas = atlas_masker.transform(new_img_like(basc444, A))
        atlas = np.argmax(atlas, axis=0)

    # # Save the transformed atlas
    np.save(os.path.join(atlas_loc, 'atlas_masked.npy'), atlas)

In [171]:
# Create a masker to standardize (0 mean, 1 SD) the image files
# and to transform them to a 2D array, as FastSRM requires
img_masker = NiftiMasker(mask_img=mask_gm, 
                              standardize=True, 
                              smoothing_fwhm=5,
                              detrend=True,
                              high_pass=1./128,
                              t_r=2.0).fit()

In [172]:
# Now create a list of movie session files 
movie_dir = os.path.join('..', task, '3mm')
subs = sorted(glob.glob(movie_dir + '/sub*'))
nsub = 0

movie_arrays = []

# Number of sessions per subject
# Different tasks have different numbers of sessions.
# Also, all subjects might not have completed all sessions.
if task == 'clips':
    # For the clips task, one subject doesn't have all 4 sessions, and
    # FastSRM requires that all subjects have the same numbers of TRs
    sessn = 3
else:
    sessn = 2

# Create 2D masked arrays from image data and save to file for quick and easy access
for s, sub in enumerate(subs):
    if os.path.isdir(sub):
        nsub += 1
        sess = sorted(glob.glob(sub + '/ses*'))
        sidx = 0
       
        for i, ses in enumerate(sess):
            if os.path.isdir(ses) and sidx < sessn:
                sidx += 1
                if os.path.exists(os.path.join(ses,'masked_imgs_preproc.npy')):
                    masked_imgs = np.load(os.path.join(ses, 'masked_imgs_preproc.npy'), 
                                          allow_pickle=True)
                else:    
                    movie_imgs = sorted(glob.glob(ses + '/' + filepattern))
                    masked_imgs = img_masker.transform(movie_imgs)
                    np.save(os.path.join(ses, 'masked_imgs_preproc.npy'), masked_imgs)

                movie_arrays.append(masked_imgs)

In [173]:
# Concatenate all the runs belonging to each subject, 
# and then create a list of lists with all subjects' data
sub_movie = []
# nsess = len(movie_arrays[0])
for i in range(0, nsub*sessn, sessn):
    part = []
    for j in range(sessn):
        # The inner concatenates create one list each for each session
        # The outer concatenate creates one list with data from all runs
        part.append(np.concatenate(movie_arrays[i+j]))
    sub_movie.append(np.concatenate(part).T)

In [174]:
# Fit the FastSRM model with the data
n_components = 20
n_jobs = 1
n_iter = 10
temp_dir = '/tmp'
low_ram = True
aggregate = 'mean'

fast_srm = FastSRM(
    atlas=atlas,
    n_components=n_components,
    n_jobs=n_jobs,
    n_iter=n_iter,
    temp_dir=temp_dir,
    low_ram=low_ram, 
    aggregate=aggregate 
)
fast_srm.fit(sub_movie)

FastSRM(aggregate='mean', atlas=array([333, 333, 190, ..., 112, 315, 315]),
        low_ram=True, n_components=20, n_iter=10, n_jobs=1, seed=None,
        temp_dir='/tmp/fastsrm0e8c3929-4c24-41c4-b837-72b735c7306a',
        verbose='warn')

In [ ]:
# Check to make sure that the spatial maps sum to zero 
# (i.e., sum of all voxel values is 0)

In [46]:
fast_srm.basis_list

['/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_0.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_1.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_2.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_3.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_4.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_5.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_6.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_7.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_8.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_9.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_10.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_11.npy',
 '/tmp/fastsrmc39d920c-f0ba-4558-9f3f-3ddaecae11e2/basis_12.npy']

In [175]:
# Load one of the basis lists to check if spatial maps sum to 0
bl1 = np.load(fast_srm.basis_list[0])
sum(bl1[0])

59.63759193783642

In [176]:
# They don't, so let us subtract the voxel mean
bl1_avg = bl1[0]-np.mean(bl1[0])

In [177]:
sum(bl1_avg)

7.768091725424142e-14

In [178]:
n_components = 20
n_voxel = len(atlas)

# Create a list of basis lists such that the resulting matrix
# is n_components x (n_voxel * n_subjects) big. This matrix will
# be the input to the spatial ICA.
# Also, the spatial maps should sum to zero.

bls = []

for s in range(nsub):
    bl_sub = np.zeros((n_components, n_voxel))
    bl = np.load(fast_srm.basis_list[s])
    for i in range(n_components):
        bl_sub[i] = bl[i] - np.mean(bl[i])
    bls.append(bl_sub)
    
basis_lists = np.concatenate(bls, axis=1)

In [121]:
# Run CanICA on the spatial maps
from nilearn.decomposition import CanICA
can_ica = CanICA(mask=mask_gm,
                 n_components=n_components,
                 standardize=True,
                )

In [ ]:
# Trying CanICA with basis_lists as input. I get an error
# because CanICA requires niimg-like objects as input.
can_ica_comp = can_ica.fit(basis_lists.T)

In [101]:
# Ok, CanICA wants list of Niimg-like objects, not 2D arrays
# Let's collect the list of basis-list nii.gz files that we've saved 

subs = sorted(glob.glob(movie_dir + '/sub*'))
nii_bl = np.empty((nsub, n_components), dtype='object')

for s, sub in enumerate(subs):
    if os.path.isdir(os.path.join(sub, 'fastsrm')):
        nii_bl[s] = np.array(sorted(glob.glob(sub + '/fastsrm/basis*gz')))

In [ ]:
# I can't use a list of lists as input, unlike for other
# nilearn functions. A single un-nested list works, however.
can_ica_res = can_ica.fit(nii_bl)

In [179]:
# Hugo reminded me that I can use FastICA, so trying 
# that. The inputs are easier here since FastICA accepts 2D
# arrays as input.

from sklearn.decomposition import FastICA

# Set up some parameters for the ICA
n_components = 20
random_state = 0
max_iter = 5000
tol = 0.005

# Initialize the ICA model
fast_ica = FastICA(n_components=n_components,
                  random_state=random_state,
                  max_iter=max_iter,
                  tol=tol)

# Transform input data using the ICA model
spatial_map_ica = fast_ica.fit_transform(basis_lists.T).T

In [180]:
# The output has dimensions n_components x (n_subjects * n_voxels)
# I checked with Hugo, and he says that the output can be treated as
# the first n_voxels of each component belong to the first subject,
# and so on. With that assumption, let's split the output.

for c in range(n_components):
    for s, sub in enumerate(subs):
        smap = spatial_map_ica[c, s*n_voxel:(s+1)*n_voxel]
        nib.save(img_masker.inverse_transform(smap), 
                     os.path.join(sub, 'fastsrm', 'sica_on_fastsrm', 
                                  'sica_basis_list-' + str(c) + '.nii.gz'))

Second level maps from sICA components

In [183]:
# Import the necessary modules/functions
import pandas as pd
import numpy as np
from nistats.second_level_model import SecondLevelModel
from nistats.thresholding import map_threshold
from nilearn import plotting
from nistats.reporting import make_glm_report

input_folder = 'sica_on_fastsrm'
input_file = 'sica_basis_list-'
subs = sorted(glob.glob(movie_dir + '/sub*'))
nsub = len(subs)

# Let's gather the files that will form the input to the second level analysis
n_components = 20
second_level_input = np.empty((nsub,n_components), dtype='object')

sidx = 0
for s, sub in enumerate(subs):
    if os.path.isdir(sub):
        for c in range(n_components):
            second_level_input[sidx][c] = os.path.join(sub, 'fastsrm', input_folder, 
                                                       input_file + str(c) + '.nii.gz')
        sidx += 1

# Construct a design matrix. We are including all subjects and 
# essentially finding the "main effects" of the contrasts performed
# in the first level analysis
design_matrix = pd.DataFrame([1] * len(second_level_input), columns=['intercept'])

# Set up the second level analysis
second_level_model = SecondLevelModel(smoothing_fwhm=8)

# Compute the contrast/main effect
for i in range(20):
    z_map = second_level_model.fit(list(second_level_input[:,i]), 
                                    design_matrix=design_matrix).compute_contrast(output_type='z_score')
    report = make_glm_report(second_level_model, 'intercept')
    report.save_as_html(os.path.join(movie_dir, 'fastsrm', input_folder, 
                                     'component-' + str(i) + '.html'))